In [14]:
# Exercise:
# Add additional subclasses and code
# to support Black-Scholes pricing of European put and call options

In [15]:
from abc import ABC, abstractmethod
import datetime
import numpy as np
import scipy.stats as stats

In [16]:
class BaseTrade:
  def __init__(self,
               ticker: str,
               side: int,
               shares: float,
               trade_date: datetime.date,
               trade_price: float) -> None:
    self.ticker = ticker
    self.side = side
    self.shares = shares
    self.trade_date = trade_date
    self.trade_price = trade_price

In [17]:
class BaseMarket:
  def __init__(self, date, spot_price) -> None:
    self.date = date
    self.spot_price = spot_price


In [18]:
class BasePricer(ABC):
  @abstractmethod
  def value(self, trade: BaseTrade, mkt: BaseMarket) -> float:
    pass

  def pnl(self, trade: BaseTrade, mkt: BaseMarket) -> float:
    if not isinstance(trade, BaseTrade):
      raise TypeError("Input must be a BaseTrade object")
    if not isinstance(mkt, BaseMarket):
      raise TypeError("Input must be a BaseMarket object")

    return self.value(trade, mkt) - trade.shares * trade.side * trade.trade_price

In [19]:
class EquityPricer(BasePricer):

  def value(self, trade: BaseTrade, mkt: BaseMarket) -> float:
    if not isinstance(trade, BaseTrade):
      raise TypeError("Input must be a BaseTrade object")
    if not isinstance(mkt, BaseMarket):
      raise TypeError("Input must be a BaseMarket object")

    return trade.shares * trade.side * mkt.spot_price

In [20]:
class EuropeanOption(BaseTrade):
  """
  European option trade.
  Inherits BaseTrade signature exactly:
    BaseTrade(ticker, side, shares, trade_date, trade_price)
  Adds:
    option_type: "C" or "P"
    strike: float
    expiry: datetime.date
  """
  def __init__(self,
               ticker: str,
               option_type: str,      # "C" or "P"
               strike: float,
               expiry: datetime.date,
               side: int,
               shares: float,
               trade_date: datetime.date,
               trade_price: float) -> None:
    super().__init__(ticker, side, shares, trade_date, trade_price)

    opt = option_type.upper()
    if opt not in ("C", "P"):
      raise ValueError("option_type must be 'C' or 'P'")

    self.option_type = opt
    self.strike = float(strike)
    self.expiry = expiry


class BlackScholesPricer(BasePricer):
  """
  Black-Scholes pricer for European call/put.
  value() returns position value = side * shares * option_price_per_share
  pnl() is inherited from BasePricer and stays consistent with EquityPricer.
  """
  def __init__(self, rate: float, vol: float) -> None:
    self.rate = float(rate)
    self.vol = float(vol)

  def _yearfrac_365(self, d1: datetime.date, d2: datetime.date) -> float:
    return (d2 - d1).days / 365.0

  def price_per_share(self, trade: EuropeanOption, mkt: BaseMarket) -> float:
    if not isinstance(trade, EuropeanOption):
      raise TypeError("Input must be a EuropeanOption object")
    if not isinstance(mkt, BaseMarket):
      raise TypeError("Input must be a BaseMarket object")

    S = float(mkt.spot_price)
    K = float(trade.strike)
    r = self.rate
    sig = self.vol

    T = self._yearfrac_365(mkt.date, trade.expiry)

    # If expired (or same day), return payoff
    if T <= 0:
      if trade.option_type == "C":
        return max(S - K, 0.0)
      else:
        return max(K - S, 0.0)

    # Defensive: vol must be positive for BS formula
    if sig <= 0:
      # fallback: discounted intrinsic-like bound (still well-defined)
      if trade.option_type == "C":
        return max(S - K * np.exp(-r * T), 0.0)
      else:
        return max(K * np.exp(-r * T) - S, 0.0)

    d1 = (np.log(S / K) + (r + 0.5 * sig**2) * T) / (sig * np.sqrt(T))
    d2 = d1 - sig * np.sqrt(T)

    if trade.option_type == "C":
      return S * stats.norm.cdf(d1) - K * np.exp(-r * T) * stats.norm.cdf(d2)
    else:
      return K * np.exp(-r * T) * stats.norm.cdf(-d2) - S * stats.norm.cdf(-d1)

  def value(self, trade: BaseTrade, mkt: BaseMarket) -> float:
    # Keep the same input-check style as EquityPricer
    if not isinstance(trade, EuropeanOption):
      raise TypeError("Input must be a EuropeanOption object")
    if not isinstance(mkt, BaseMarket):
      raise TypeError("Input must be a BaseMarket object")

    px = self.price_per_share(trade, mkt)
    return trade.shares * trade.side * px


In [21]:
# test
value_date = datetime.date(2026, 1, 30)
# AAPL equity position
side, shares, trade_price = 1, 1000, 258.05
trade_date = datetime.date(2026, 1, 15)
trade = BaseTrade("AAPL", side, shares, value_date, trade_price)
# AAPL market
mkt = BaseMarket(value_date, 255)
# equity pricer
eq_pricer = EquityPricer()
eq_pricer.value(trade, mkt), eq_pricer.pnl(trade, mkt)

(255000, -3050.0)

In [ ]:

value_date = datetime.date(2026, 1, 30)


mkt = BaseMarket(value_date, 255)


side, shares, trade_price = 1, 1000, 5.00
trade_date = datetime.date(2026, 1, 15)

opt = EuropeanOption(
    "AAPL",                   # ticker / underlying
    "C",                      # "C" or "P"
    260,                      # strike
    datetime.date(2026, 6, 30),# expiry
    side,
    shares,
    trade_date,
    trade_price
)

bs = BlackScholesPricer(rate=0.05, vol=0.25)

bs.value(opt, mkt), bs.pnl(opt, mkt), bs.price_per_share(opt, mkt)


(np.float64(16492.055064064232),
 np.float64(11492.055064064232),
 np.float64(16.492055064064232))